# Command Method Design Pattern:

## Some Use Cases:
1. Batch Data Processing Jobs:
- Use Case: In batch processing systems, where large amounts of data are processed in scheduled jobs, the Command pattern can be used to encapsulate each batch job (e.g., data transformation, loading, or aggregation) as a command. This allows for flexible job scheduling and execution.
- Benefit: The Command pattern enables modularity by separating the logic of the jobs from the scheduling mechanism, allowing for easy extension and reusability of individual data processing tasks.
2. Data Export and Import Automation:
- Use Case: When data needs to be exported from or imported into different systems (databases, APIs, file systems), the Command pattern can encapsulate each export/import action as a command. This allows users to queue, execute, and undo these actions easily.
- Benefit: The Command pattern provides a structured way to manage various data export/import operations, enabling easy tracking, undoing, and automating complex workflows.
3. Data Transformation Workflow Control:
- Use Case: In data engineering, when multiple transformations need to be applied to a dataset in a specific order (e.g., cleaning, filtering, enrichment), the Command pattern can be used to encapsulate each transformation step as a command.
- Benefit: The Command pattern allows for better control and flexibility over the sequence of transformations, enabling easy addition, removal, or reordering of data transformation steps without altering the core logic.

## 1. Scenario: Managing Database Operations
- You are building a system to manage database operations such as INSERT, UPDATE, DELETE, and SELECT. The operations can be queued, executed later, or even rolled back (undo feature).

#### The challenge is to design a system where:
- The system is not tightly coupled with the database handling logic.
New operations (e.g., bulk insert, batch updates) can be added without modifying existing code.
Undo/redo of operations is easy to implement.

### 1.2 Solution using Traditional Approach:

In [3]:
class Database:
    def __init__(self):
        self.history = []

    def insert(self, record):
        print(f"Inserting {record}")
        self.history.append(("insert", record))

    def update(self, record):
        print(f"Updating {record}")
        self.history.append(("update", record))

    def delete(self, record):
        print(f"Deleting {record}")
        self.history.append(("delete", record))

    def undo_last_action(self):
        if not self.history:
            print("No actions to undo.")
            return
        
        action, record = self.history.pop()
        if action == "insert":
            print(f"Undoing insert: Deleting {record}")
        elif action == "update":
            print(f"Undoing update on {record}")
        elif action == "delete":
            print(f"Undoing delete: Re-inserting {record}")


# Usage
db = Database()
db.insert("Record1")  # Inserting Record1
db.update("Record1")  # Updating Record1
db.delete("Record1")  # Deleting Record1
db.undo_last_action()  # Undoing delete: Re-inserting Record1
db.undo_last_action()  # Undoing update on Record1


Inserting Record1
Updating Record1
Deleting Record1
Undoing delete: Re-inserting Record1
Undoing update on Record1


### Problems with the Traditional Method:
- Tight Coupling: The Database class handles both executing operations and maintaining history, violating the Single Responsibility Principle.
- Scalability Issues: Adding new operations requires modifying the Database class, leading to poor extensibility.
- No Abstraction for Operations: Operations (insert, update, delete) are hardcoded in the undo_last_action logic, making it difficult to adapt or extend.
- Hard to Manage Complex Sequences: Undoing or redoing specific commands is cumbersome as all logic resides within the Database.

### 1.2 Solution Using Command Design Pattern:
- With the Command Design Pattern, each database operation is encapsulated as a Command object. The client queues and executes commands, decoupling itself from the database.

### Components of Command Design Pattern:

- Command: A blueprint defining the action to be performed and a way to undo it.
- Concrete Command: Specific implementation of an action, often linked to the object performing it.
- Receiver: The entity that carries out the actual work or task when a command is executed.
- Invoker: The handler or manager that triggers the execution of commands and may store them for future use (e.g., undo/redo).
- Client: The originator that decides the actions, creates the commands, and connects them to the invoker.

In [2]:
# Command Interface
class Command:
    def execute(self):
        pass

    def undo(self):
        pass


# Concrete Commands
class InsertCommand(Command):
    def __init__(self, database, record):
        self.database = database
        self.record = record

    def execute(self):
        self.database.insert(self.record)

    def undo(self):
        self.database.delete(self.record)


class UpdateCommand(Command):
    def __init__(self, database, record):
        self.database = database
        self.record = record

    def execute(self):
        self.database.update(self.record)

    def undo(self):
        print(f"Reverting update on {self.record}")


class DeleteCommand(Command):
    def __init__(self, database, record):
        self.database = database
        self.record = record

    def execute(self):
        self.database.delete(self.record)

    def undo(self):
        self.database.insert(self.record)


# Invoker
class CommandManager:
    def __init__(self):
        self.command_queue = []
        self.history = []

    def execute_command(self, command):
        command.execute()
        self.history.append(command)

    def undo_last_command(self):
        if self.history:
            last_command = self.history.pop()
            last_command.undo()

# Database Class (Receiver)
class Database:
    def insert(self, record):
        print(f"Inserting {record}")

    def update(self, record):
        print(f"Updating {record}")

    def delete(self, record):
        print(f"Deleting {record}")


# Usage
db = Database()
manager = CommandManager()

# Create commands
insert_cmd = InsertCommand(db, "Record1")
update_cmd = UpdateCommand(db, "Record1")
delete_cmd = DeleteCommand(db, "Record1")

# Execute commands/ passing request as an object
manager.execute_command(insert_cmd)  # Inserting Record1
manager.execute_command(update_cmd)  # Updating Record1
manager.execute_command(delete_cmd)  # Deleting Record1

# Undo last command
manager.undo_last_command()  # Reverts deletion: Inserting Record1


Inserting Record1
Updating Record1
Deleting Record1
Inserting Record1


### How Command Pattern Solves These Problems:

- Encapsulation of Commands: Each command (InsertCommand, UpdateCommand, DeleteCommand) is encapsulated in a class, separating execution logic from the Database.
- Extensibility: Adding new operations is as simple as creating a new Command class without modifying existing code.
- History Management: The CommandManager independently handles the history of commands, adhering to Single Responsibility Principle.
- Reusability: Commands are reusable and can be executed or undone without duplicating logic.
- Dynamic Behavior: You can add, modify, or remove commands dynamically during runtime.

### Advantage of decoupling of Invoker and Receiver:
- Separation of Concerns: Invoker doesn’t need to know the execution logic (Receiver).
- Extensibility: Easily add new commands without altering existing code.
- Undo/Redo Support: Manage actions with undo functionality.
- Flexibility: Reuse invoker with different receivers.

### Advantage of passing Request as an Object:
- Decoupling Sender and Receiver: The sender only knows the command exists, not how it's executed, reducing dependencies.
- Flexibility: Commands can be queued, stored, or passed around easily for scheduling, undo/redo, or batch processing.
- Undo/Redo: Commands can store enough info to allow for easy reversal or re-execution of actions.
- Maintainability: Changes to logic are confined to the command object, reducing risk in the rest of the system.
- Reusability: Command objects can be reused across different parts of the application.
- Extensibility: New commands can be added without altering existing code.
- Auditability: Commands can encapsulate logging, validation, or auditing, keeping concerns separate.
- Parameterization: Commands store all necessary data, allowing for more focused and clean execution